<a href="https://colab.research.google.com/github/acgowda/marketflow/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from plotly import express as px
import matplotlib as mpl

mpl.style.use('seaborn')

In [33]:
tsla = yf.Ticker('TSLA')
df = tsla.history(start = '2000-1-1',end = '2022-4-27')

In [34]:
df.head(20)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0
2010-07-07,3.280,3.326,2.996,3.160,34608500,0,0.0
2010-07-08,3.228,3.504,3.114,3.492,38557000,0,0.0
2010-07-09,3.516,3.580,3.310,3.480,20253000,0,0.0
2010-07-12,3.590,3.614,3.400,3.410,11012500,0,0.0


In [ ]:
tsla.financials

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,2593000000.0,1491000000.0,1343000000.0,1460000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,6343000000.0,1154000000.0,-665000000.0,-1005000000.0
Minority Interest,1394000000.0,1454000000.0,1492000000.0,1390000000.0
Net Income,5519000000.0,721000000.0,-862000000.0,-976000000.0
Selling General Administrative,4517000000.0,3188000000.0,2646000000.0,2835000000.0
Gross Profit,13606000000.0,6630000000.0,4069000000.0,4042000000.0
Ebit,6523000000.0,1951000000.0,80000000.0,-253000000.0
Operating Income,6523000000.0,1951000000.0,80000000.0,-253000000.0
Other Operating Expenses,-27000000.0,-27000000.0,-27000000.0,-27000000.0


In [ ]:
#you're listening too hard bro
# https://medium.com/automated-trading/obtain-40-technical-indicators-for-a-stock-using-python-247b32e85f30

In [36]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"]), dict(values=dt_breaks)])

# add moving averages to df
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA5'] = df['Close'].rolling(window=5).mean()

fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))

fig.show()

In [ ]:
import random
from collections import deque
from sklearn import preprocessing

In [ ]:
seq_len = 60  # how long of a preceeding sequence to collect for RNN
step_ahead = 3  # how far into the future are we trying to predict?
target_ratio = "TSLA"

def classify(current, future):
    if float(future) > float(current):  # if the future price is higher than the current, that's a buy, or a 1
        return 1
    else:  # otherwise... it's a 0!
        return 0


def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.


    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!